# Защита персональных данных клиентов

## Вступление

**Описание:**

В нашем распоряжение есть личные данных клиентов страховой компании. Необходимо защитить данные клиентов, разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы. Также нужно защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

**Цель:** Найти и обосновать способ шифрования персональных данных без потери качества модели машинного обучения.

**Задачи:**

- Загрузитб и изучите данные.
- Ответить на вопрос и обосновать решение: "Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)"<br>
        a. Изменится. Приведите примеры матриц.<br>
        b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.
- Предложить алгоритм преобразования данных для решения задачи. Обосновать, почему качество линейной регрессии не поменяется.
- Запрограммировать этот алгоритм, применив матричные операции. Проверить, что качество линейной регрессии из sklearn не отличается до и после преобразования. Применить метрику R2.

**Описание данных:**

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import random as r

In [2]:
try:
    data = pd.read_csv('insurance.csv')
except FileNotFoundError:
    data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
data['Возраст'].value_counts().head(7)

19.0    223
25.0    214
31.0    212
26.0    211
22.0    209
27.0    209
32.0    206
Name: Возраст, dtype: int64

In [7]:
data['Зарплата'].value_counts().head(7)

45800.0    29
37100.0    28
41500.0    27
43200.0    27
46800.0    26
38800.0    26
32800.0    26
Name: Зарплата, dtype: int64

Данные в целом хорошие. Пропусков нет. Единственно, что прям смущает немного это тип данных у возраста и зарплаты - стоят вещественный тип. Но по факту у нас только целые значения в обоих признаках. Но думаю это не критично.

по количественным данным видим, что на первый взгляд данные распределны нормально. медиана и среднее значение относительно близко к друг другу находятся по значения. Каких то диких выбрсоов не наблюдается

### 2. Если признаки умножить на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
a. Изменится. Приведите примеры матриц.<br>
b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

Мой ответ, нет не изменится.

Если у нас есть матрциа с признаками $Х$ размерностью $m x n$ и обратимая матрица $Z$ размерностью $n x n$. После перемножения мы получим матрицу $Х2$ размерностью так же $m x n$ соотвественно.

Теперь запишем уравнение линейной регрессии для обоих случаев:

$y = X*w + w0$, $y2 = X2*w2 + w02$.

так как $Х2 = Х * Z$, то выразив $X2$ получаем следующее уравнение: $y2 = X*Z*w2 + w02$. 

Теперь сравним обе линейные регрессии. 

Для этого используем формулу для обучения линейной регрессии:
$w=(X^{T}*X)^{-1}*X^{T}*y$

Теперь представим, что мы умножили матрицу признаков $X$ на обратимую матрицу $Z$, получив $X2 = XZ$. Подставим $X2$ в формулу обучения:
$w2=((X*Z)^{T}*X*Z)^{−1}(X*Z)^{T}*y$

по правилу $(AB)^{T} = B^{T} * A ^{T}$ получаем:

$w2=(Z^{T}*X^{T}*X*Z)^{−1}*Z^{T}*X^{T}*y$

по правилу $(A*B)^{-1}=B^{-1}*A^{-1}$ получаем:

$w2=Z^{−1}*(X^{T}*X)^{−1}*(Z^{T})^{−1}*Z^{T}*X^{T}*y$

Так как $A * A^{-1} = E$, где $E$ - единичная матрица, а $A$ - любая обратимая матрциа, то получаем:

$w2=Z^{−1}*(X^{T}*X)^{−1}* E * X^{T}*y$, и при умножении матрицы на единчную матрицу - поулчаем исходную матрицу, то затем вспоминаем, что $w=(X^{T}*X)^{−1}*X^{T}*y$, подставим.

Итак, мы видим, что $w2 = Z^{-1} w$, $w02 = w0$.

Таким образом, когда мы умножаем матрицу признаков на обратимую матрицу, параметры модели линейной регрессии также изменяются, но взаимосвязи между признаками и целевой переменной сохраняются, что позволяет сохранить качество линейной регрессии.

Теперь эти закономерности подставим заново в $y2$:

$y2 = X * Z * Z^{-1} * w + w0 = X * E * w + w0 = y$, где $Е$ - единчная матрциа, получающая при перемножении матрицы на ее обратную. Проверим.

In [8]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

# data_train, data_test = train_test_split(data, test_size=0.7, random_state=12345)
# X_train = data_train.drop('Страховые выплаты', axis=1)
# Y_train = data_train['Страховые выплаты']
# X_test = data_test.drop('Страховые выплаты', axis=1)
# Y_test = data_test['Страховые выплаты']
    

Создадим функцию для создания рандомной обратимой матрицы необходимой размерности

In [9]:
def gen_randinvert_matrix(n):
    while True:
        random_matrix = np.random.rand(n, n) # случайная матрица размером n x n с числами из [0, 1)
        if np.linalg.det(random_matrix) != 0: # проверка на обратимость
            return random_matrix
        
randinvert_matrix = gen_randinvert_matrix(features.shape[1])
print(randinvert_matrix)


[[0.53590978 0.48459166 0.32356823 0.39815133]
 [0.66429819 0.05390658 0.33387405 0.3529248 ]
 [0.97937039 0.16643406 0.59457524 0.71997095]
 [0.59229971 0.38125375 0.11553582 0.51591593]]


Создадим функцию для рассчета линейной регрессии без обратимой матрицы. Сразу же определим качество модели с помощью метрики R2

In [10]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [11]:
model_1 = LinearRegression()
model_1.fit(features, target)
predictions_1 = model_1.predict(features)
r2_1 = r2_score(target, predictions_1)
print(r2_1)

0.4249455028666801


Делаем тоже самое, но с добавлением обратимой матрциы. Для этого преобразуем сразу наши признаки путем умножения на рандомную обратимую матрицу.

In [12]:
features_transformed = features.dot(randinvert_matrix)

Далее делаем то же самое, используем нашу функцию, обучаем, предсказываем и считаем R2

In [13]:
model_2 = LinearRegression()
model_2.fit(features_transformed, target)
predictions_2 = model_2.predict(features_transformed)
r2_2 = r2_score(target, predictions_2)
print(r2_2)

0.4249455028623569


In [14]:
print(abs(r2_1 - r2_2))

4.3232084578903596e-12


Что мы получаем, по модулю разница метрик R2 полученных при разных условиях (без умножения на обратимую марицу и с умножением) - ничтожна мала. (рандомное значение) на 10 в -13 степени. Такая погрешность могла возникнуть по нескольким причинам: 1 - например изза округления чисел при вычислениях (численные апроксимации), 2 - возможно наличие какогото шума в данных.

Соотвественно, чтобы дешифровать данные - нужно умножить зашифрованные данные на обратную матрицу нашей обратимой матрицы.

X * M * M(-1) = X * E = X

обратимая матрциа умножаясь на ее обратную даст "едничную", а произведение единичной на нашу исходную дает просто исходную. То же самое что: 5 * 1 = 5

In [15]:
features_transformed_back = np.round((features_transformed.dot(np.linalg.inv(randinvert_matrix))))

In [16]:
features_transformed_back

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,0.0
...,...,...,...,...
4995,-0.0,28.0,35700.0,2.0
4996,0.0,34.0,52400.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


In [17]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


Из- за делений и умножений получаются не очень приятные данные и не "чистый ноль", там где он должен быть. поэтому пришлось округлить данные для целого, чтобы получить наши первоначальные целые значения.

Зашифровать данные клиентов можно используя обратимую матрицу, с рандомными значениями. Как это было сделано/показано/доказано в предыдущем пункте.

Умножение признаков на значения обратимой марицы не меняет линейную зависиомсть между признаки и целевыми значениями. В качестве формулы доказательство тоже было в пункте 2. Даже после умножения признаков на обратимую матрицу, линейная регрессия все равно  предсказывает с тем же качеством, что и в исходной модели без умножения. Поэтому метрики качества получаются приближенно равными. 

## Вывод:

Выяснили, разобрались и доказали, что качество линейной регрессии на первоначальных признаках и на зашифрованных признаках (путем домножения рандомной обратимой матрицы), можно считать, что никак не меняется. Опробовали эту "теорему" на наших исодных личных данных страховой компании. Получили значение метрики R2 линейной регресиси на исходных данных = 0.4249455028666801 и R2 линейной регресиси на зашифрованных данных = 0.4249455028662076. Разница по модулю составила - 4.7 на 10 в -13 степени. Что можно считать ничтожно малым изменением, потому сочли, что качество модели от домножения на обратимую матрицу - не страдает. 

Ну, а чтобы дешифровать исходные данные, достаточно умножить зашифрованные данные на обратную матрицу нашей обратимой матрицы. Если значения целые, то стоит еще использовать округление до целого, чтобы не испугаться результата деления=) Получается как и всегда, чтобы дешифровать данные - необходимо значит ключ шифрования. Возможно, если бы Цезарь знал ЛАГ, то возможно придумал бы что-то посложнее для своих шифров))) Ибо в нашем случае, чтобы расшифровать все данные необходимо знать все значения обратимой матрицы с максимальной точностью и правильной размерности - иначе ничего не выйдет или получаться искаженные значения.